In [71]:
import os
import datetime
from tqdm import tqdm
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from championWrapper import ChampionWrapper

In [72]:
in_data = pd.read_csv('pb_table.csv', index_col=False)
champs = ChampionWrapper()

num_champs = len(champs.champToID)
num_champs


/Users/timinzitari/miniforge3/envs/tensorflow_m1/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


158

In [73]:
#isolate the input frames of hero choices by name
isolated_input = in_data[['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5']]
isolated_input.head()

,BB1,RB1,BB2,RB2,BB3,RB3,BP1,RP1,RP2,BP2,BP3,RP3,RB4,BB4,RB5,BB5,RP4,BP4,BP5,RP5
0,Lulu,Pantheon,Senna,Renekton,Olaf,Camille,Udyr,Twisted Fate,Kai'Sa,Samira,Azir,Taliyah,Rell,Alistar,Gragas,Jayce,Leona,Nautilus,Gangplank,Irelia
1,Lulu,Pantheon,Senna,Renekton,Olaf,Camille,Taliyah,Kai'Sa,Twisted Fate,Samira,Rell,Udyr,Orianna,Alistar,Azir,Nautilus,Leona,Gnar,Syndra,Irelia
2,Renekton,Pantheon,Camille,Lulu,Udyr,Seraphine,Olaf,Thresh,Graves,Senna,Tahm Kench,Aphelios,Twisted Fate,Azir,Karma,Orianna,Syndra,Zoe,Malphite,Gangplank
3,Renekton,Pantheon,Camille,Lulu,Thresh,Twisted Fate,Olaf,Udyr,Kai'Sa,Senna,Tahm Kench,Azir,Orianna,Irelia,Syndra,Jayce,Gnar,Karma,LeBlanc,Blitzcrank
4,Lulu,Pantheon,Lillia,Renekton,Olaf,Rell,Taliyah,Camille,Twisted Fate,Jax,Galio,Udyr,Samira,Senna,Thresh,Alistar,Kai'Sa,Xayah,Leona,Sett


In [74]:
#Dataframe to Numpy
inputnpArray = isolated_input.to_numpy()
inputnpArray


array([['Lulu', 'Pantheon', 'Senna', ..., 'Nautilus', 'Gangplank',
        'Irelia'],
       ['Lulu', 'Pantheon', 'Senna', ..., 'Gnar', 'Syndra', 'Irelia'],
       ['Renekton', 'Pantheon', 'Camille', ..., 'Zoe', 'Malphite',
        'Gangplank'],
       ...,
       ['Renekton', 'Ziggs', 'Thresh', ..., 'Volibear', 'Rakan', 'Braum'],
       ['Ryze', 'LeBlanc', 'Tristana', ..., 'Sejuani', 'Jayce',
        'Cassiopeia'],
       ['Ryze', 'Kennen', 'Tristana', ..., 'Camille', 'Kindred', 'Jayce']],
      dtype=object)

In [75]:
# Numpy convert Names to Ids
for match in range(len(inputnpArray)):
    for column in range(len(inputnpArray[match])):
        inputnpArray[match][column] = champs.cNameToId(inputnpArray[match][column])

inputnpArray



array([[98, 71, 127, ..., 93, 41, 39],
       [98, 71, 127, ..., 114, 107, 39],
       [54, 71, 120, ..., 110, 50, 41],
       ...,
       [54, 97, 139, ..., 90, 145, 121],
       [13, 7, 18, ..., 95, 103, 63],
       [13, 76, 18, ..., 120, 123, 103]], dtype=object)

Here we have the data   
In Raw form we want to make it into [In][Label] sets to    
use in our Sequence to Sequence model   

In [76]:
#Here we have the data
# In Raw form we want to make it into [In][Label] sets to 
# use in our Sequence to Sequence model

In [77]:
numberpd = pd.DataFrame(inputnpArray, columns=['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5'])
print(numberpd)

      BB1 RB1  BB2  RB2  BB3  RB3  BP1  RP1  RP2  BP2  BP3  RP3  RB4  BB4  \
0      98  71  127   54    2  120   68    4  112  138  136  118  149   12   
1      98  71  127   54    2  120  118  112    4  138  149   68   57   12   
2      54  71  120   98   68  113    2  139   88  127  125  151    4  136   
3      54  71  120   98  139    4    2   68  112  127  125  136   57   39   
4      98  71  157   54    2  149  118  120    4   24    3   68  138  127   
...   ...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
4073    4  13    7   46    5  139   60  126   78  121  151   97  156  103   
4074  139  97   54   60    4   13    7  120  104  103   46  126   92  151   
4075   54  97  139   60  123   39   13    4  120   92   96    5   46  151   
4076   13   7   18  139   92   76   97   39   55  120  144  152   57   75   
4077   13  76   18   41   97   39    7  139  136   92  121   55   46   22   

      RB5  BB5  RP4  BP4  BP5  RP5  
0      70  103   78   93   41   39  
1

In [78]:
traindf, testdf = train_test_split(numberpd, test_size=0.2)
testdf, valdf = train_test_split(testdf, test_size=0.5)

In [79]:
tfl = tf.keras.layers
n_features=num_champs
results = {}
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)
the_loss = 'mse'
num_epochs=500
from WindowGenerator import SplitGen

splitter = SplitGen()
train, test = train_test_split(inputnpArray, test_size=0.2)
test, val = train_test_split(test, test_size=0.5)

i=18
o=2
f=num_champs

X_train, y_train = splitter.split_sequences(train, i, o, f)
X_test, y_test = splitter.split_sequences(test, i, o, f)
X_val, y_val = splitter.split_sequences(val, i, o, f)


In [80]:
doubleLSTM_model = tf.keras.Sequential([
    tfl.LSTM(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.LSTM(256, activation='relu', return_sequences=True),
    tfl.TimeDistributed(tfl.Dense(32)),
    tfl.TimeDistributed(tfl.Dense(f))
])


doubleLSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

doubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['doubleLSTM']=doubleLSTM_model.evaluate(X_test, y_test)
print(results['doubleLSTM'])



Epoch 1/500
102/102 [==============================] - 6s 48ms/step - loss: 0.0062 - accuracy: 0.0372 - val_loss: 0.0062 - val_accuracy: 0.0404
Epoch 2/500
102/102 [==============================] - 5s 48ms/step - loss: 0.0062 - accuracy: 0.0474 - val_loss: 0.0062 - val_accuracy: 0.0417
Epoch 3/500
102/102 [==============================] - 5s 47ms/step - loss: 0.0062 - accuracy: 0.0601 - val_loss: 0.0062 - val_accuracy: 0.0600
Epoch 4/500
102/102 [==============================] - 5s 51ms/step - loss: 0.0062 - accuracy: 0.0702 - val_loss: 0.0062 - val_accuracy: 0.0662
Epoch 5/500
102/102 [==============================] - 5s 48ms/step - loss: 0.0062 - accuracy: 0.0835 - val_loss: 0.0062 - val_accuracy: 0.0711
Epoch 6/500
102/102 [==============================] - 5s 47ms/step - loss: 0.0061 - accuracy: 0.0898 - val_loss: 0.0062 - val_accuracy: 0.0784
Epoch 7/500
102/102 [==============================] - 5s 48ms/step - loss: 0.0061 - accuracy: 0.0983 - val_loss: 0.0062 - val_accuracy:

In [81]:
bidoubleLSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.Bidirectional(tfl.LSTM(256, activation='relu', return_sequences=True)),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f,activation='sigmoid'))
])
bidoubleLSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

bidoubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['bidoubleLSTM']=bidoubleLSTM_model.evaluate(X_test, y_test)
print(results['bidoubleLSTM'])

Epoch 1/500
102/102 [==============================] - 10s 82ms/step - loss: 0.0433 - accuracy: 0.0017 - val_loss: 0.0063 - val_accuracy: 0.0000e+00
Epoch 2/500
102/102 [==============================] - 8s 79ms/step - loss: 0.0063 - accuracy: 0.0000e+00 - val_loss: 0.0063 - val_accuracy: 0.0000e+00
Epoch 3/500
102/102 [==============================] - 8s 80ms/step - loss: 0.0063 - accuracy: 0.0000e+00 - val_loss: 0.0063 - val_accuracy: 0.0000e+00
Epoch 4/500
102/102 [==============================] - 8s 79ms/step - loss: 0.0063 - accuracy: 0.0000e+00 - val_loss: 0.0063 - val_accuracy: 0.0000e+00
Epoch 5/500
102/102 [==============================] - 8s 80ms/step - loss: 0.0063 - accuracy: 0.0000e+00 - val_loss: 0.0063 - val_accuracy: 0.0000e+00
Epoch 6/500
102/102 [==============================] - 8s 80ms/step - loss: 0.0063 - accuracy: 0.0000e+00 - val_loss: 0.0063 - val_accuracy: 0.0000e+00
Epoch 7/500
102/102 [==============================] - 8s 79ms/step - loss: 0.0063 - accura

In [82]:
hydoubleLSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.LSTM(256, activation='relu', return_sequences=True),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f))
])
hydoubleLSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

hydoubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['hydoublelstm']=hydoubleLSTM_model.evaluate(X_test, y_test)
print(results['hydoublelstm'])

Epoch 1/500
102/102 [==============================] - 9s 82ms/step - loss: 0.0062 - accuracy: 0.0383 - val_loss: 0.0062 - val_accuracy: 0.0466
Epoch 2/500
102/102 [==============================] - 8s 78ms/step - loss: 0.0062 - accuracy: 0.0504 - val_loss: 0.0062 - val_accuracy: 0.0564
Epoch 3/500
102/102 [==============================] - 8s 83ms/step - loss: 0.0062 - accuracy: 0.0739 - val_loss: 0.0062 - val_accuracy: 0.0699
Epoch 4/500
102/102 [==============================] - 8s 82ms/step - loss: 0.0062 - accuracy: 0.0771 - val_loss: 0.0062 - val_accuracy: 0.0686
Epoch 5/500
102/102 [==============================] - 8s 77ms/step - loss: 0.0061 - accuracy: 0.0861 - val_loss: 0.0062 - val_accuracy: 0.0711
Epoch 6/500
102/102 [==============================] - 8s 77ms/step - loss: 0.0061 - accuracy: 0.1025 - val_loss: 0.0062 - val_accuracy: 0.0784
Epoch 7/500
102/102 [==============================] - 8s 76ms/step - loss: 0.0061 - accuracy: 0.1151 - val_loss: 0.0062 - val_accuracy:

In [83]:
lSTM_model = tf.keras.Sequential([
    tfl.LSTM(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f))
])
lSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

lSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['lstm']=lSTM_model.evaluate(X_test, y_test)
print(results['lstm'])

Epoch 1/500
102/102 [==============================] - 5s 45ms/step - loss: 0.0062 - accuracy: 0.0532 - val_loss: 0.0062 - val_accuracy: 0.0650
Epoch 2/500
102/102 [==============================] - 5s 46ms/step - loss: 0.0062 - accuracy: 0.0768 - val_loss: 0.0062 - val_accuracy: 0.0723
Epoch 3/500
102/102 [==============================] - 5s 46ms/step - loss: 0.0062 - accuracy: 0.0906 - val_loss: 0.0062 - val_accuracy: 0.0723
Epoch 4/500
102/102 [==============================] - 4s 43ms/step - loss: 0.0061 - accuracy: 0.0996 - val_loss: 0.0062 - val_accuracy: 0.0772
Epoch 5/500
102/102 [==============================] - 4s 43ms/step - loss: 0.0061 - accuracy: 0.1117 - val_loss: 0.0062 - val_accuracy: 0.0870
Epoch 6/500
102/102 [==============================] - 4s 42ms/step - loss: 0.0061 - accuracy: 0.1188 - val_loss: 0.0062 - val_accuracy: 0.0858
Epoch 7/500
102/102 [==============================] - 4s 43ms/step - loss: 0.0061 - accuracy: 0.1292 - val_loss: 0.0062 - val_accuracy:

In [84]:
rnn_model = tf.keras.Sequential([
    tfl.SimpleRNN(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.SimpleRNN(256, activation='relu', return_sequences=True),
    tfl.TimeDistributed(tfl.Dense(128, activation='relu')),
    tfl.TimeDistributed(tfl.Dense(f))
])
rnn_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

rnn_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['rnn']=rnn_model.evaluate(X_test, y_test)
print(results['rnn'])

Epoch 1/500
102/102 [==============================] - 2s 19ms/step - loss: 0.0063 - accuracy: 0.0351 - val_loss: 0.0062 - val_accuracy: 0.0466
Epoch 2/500
102/102 [==============================] - 2s 23ms/step - loss: 0.0062 - accuracy: 0.0561 - val_loss: 0.0062 - val_accuracy: 0.0600
Epoch 3/500
102/102 [==============================] - 2s 22ms/step - loss: 0.0062 - accuracy: 0.0707 - val_loss: 0.0062 - val_accuracy: 0.0600
Epoch 4/500
102/102 [==============================] - 2s 19ms/step - loss: 0.0062 - accuracy: 0.0837 - val_loss: 0.0062 - val_accuracy: 0.0576
Epoch 5/500
102/102 [==============================] - 2s 19ms/step - loss: 0.0061 - accuracy: 0.1097 - val_loss: 0.0062 - val_accuracy: 0.0662
Epoch 6/500
102/102 [==============================] - 2s 19ms/step - loss: 0.0060 - accuracy: 0.1341 - val_loss: 0.0062 - val_accuracy: 0.0723
Epoch 7/500
102/102 [==============================] - 2s 20ms/step - loss: 0.0059 - accuracy: 0.1784 - val_loss: 0.0063 - val_accuracy:

In [85]:
singrnn_model = tf.keras.Sequential([
    tfl.SimpleRNN(256, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.TimeDistributed(tfl.Dense(128, activation='relu')),
    tfl.TimeDistributed(tfl.Dense(f))
])
singrnn_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

singrnn_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['singrnn']=singrnn_model.evaluate(X_test, y_test)
print(results['singrnn'])

Epoch 1/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0063 - accuracy: 0.0331 - val_loss: 0.0062 - val_accuracy: 0.0441
Epoch 2/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0062 - accuracy: 0.0422 - val_loss: 0.0062 - val_accuracy: 0.0417
Epoch 3/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0062 - accuracy: 0.0443 - val_loss: 0.0062 - val_accuracy: 0.0429
Epoch 4/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0062 - accuracy: 0.0538 - val_loss: 0.0062 - val_accuracy: 0.0368
Epoch 5/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0062 - accuracy: 0.0670 - val_loss: 0.0062 - val_accuracy: 0.0392
Epoch 6/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0062 - accuracy: 0.0903 - val_loss: 0.0062 - val_accuracy: 0.0429
Epoch 7/500
102/102 [==============================] - 1s 8ms/step - loss: 0.0061 - accuracy: 0.1030 - val_loss: 0.0062 - val_accuracy: 0.0490

In [86]:
bi_LSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f))
])
bi_LSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

bi_LSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['bilstm']=bi_LSTM_model.evaluate(X_test, y_test)
print(results['bilstm'])

Epoch 1/500
102/102 [==============================] - 9s 79ms/step - loss: 0.0062 - accuracy: 0.0441 - val_loss: 0.0062 - val_accuracy: 0.0735
Epoch 2/500
102/102 [==============================] - 8s 79ms/step - loss: 0.0062 - accuracy: 0.0865 - val_loss: 0.0062 - val_accuracy: 0.0723
Epoch 3/500
102/102 [==============================] - 7s 73ms/step - loss: 0.0061 - accuracy: 0.1016 - val_loss: 0.0062 - val_accuracy: 0.0674
Epoch 4/500
102/102 [==============================] - 8s 75ms/step - loss: 0.0061 - accuracy: 0.1156 - val_loss: 0.0062 - val_accuracy: 0.0858
Epoch 5/500
102/102 [==============================] - 9s 84ms/step - loss: 0.0061 - accuracy: 0.1257 - val_loss: 0.0062 - val_accuracy: 0.0809
Epoch 6/500
102/102 [==============================] - 8s 74ms/step - loss: 0.0060 - accuracy: 0.1426 - val_loss: 0.0062 - val_accuracy: 0.0956
Epoch 7/500
102/102 [==============================] - 7s 72ms/step - loss: 0.0060 - accuracy: 0.1560 - val_loss: 0.0062 - val_accuracy:

In [87]:
CONV_WIDTH = 3
conv_model = tf.keras.Sequential([

 tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(CONV_WIDTH)),
    tf.keras.layers.Flatten(),
    tfl.Dense(128),
    tf.keras.layers.Dense(o*f,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([o, f])

])
conv_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

conv_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----\n")
results['conv']=conv_model.evaluate(X_test, y_test)
print(results['conv'])

Epoch 1/500
102/102 [==============================] - 0s 1ms/step - loss: 0.0062 - accuracy: 0.0392 - val_loss: 0.0062 - val_accuracy: 0.0576
Epoch 2/500
102/102 [==============================] - 0s 783us/step - loss: 0.0062 - accuracy: 0.0769 - val_loss: 0.0062 - val_accuracy: 0.0674
Epoch 3/500
102/102 [==============================] - 0s 772us/step - loss: 0.0061 - accuracy: 0.1102 - val_loss: 0.0062 - val_accuracy: 0.0662
Epoch 4/500
102/102 [==============================] - 0s 771us/step - loss: 0.0061 - accuracy: 0.1360 - val_loss: 0.0062 - val_accuracy: 0.0772
Epoch 5/500
102/102 [==============================] - 0s 751us/step - loss: 0.0060 - accuracy: 0.1504 - val_loss: 0.0062 - val_accuracy: 0.0895
Epoch 6/500
102/102 [==============================] - 0s 769us/step - loss: 0.0060 - accuracy: 0.1662 - val_loss: 0.0063 - val_accuracy: 0.0797
Epoch 7/500
102/102 [==============================] - 0s 757us/step - loss: 0.0059 - accuracy: 0.1806 - val_loss: 0.0063 - val_accu

In [88]:
clstm_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(CONV_WIDTH)),
    tfl.MaxPooling1D(pool_size=2),
    tfl.Flatten(),
    tfl.RepeatVector(o),
    tfl.LSTM(512, activation='relu', return_sequences=True), 
    tfl.TimeDistributed(tfl.Dense(256,activation='relu')),
    tfl.TimeDistributed(tfl.Dense(f))
])

clstm_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

clstm_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\neval-----\n")
results['clstm']=clstm_model.evaluate(X_test, y_test)
print(results['clstm'])

Epoch 1/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0062 - accuracy: 0.0359 - val_loss: 0.0062 - val_accuracy: 0.0368
Epoch 2/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0062 - accuracy: 0.0478 - val_loss: 0.0062 - val_accuracy: 0.0588
Epoch 3/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0062 - accuracy: 0.0808 - val_loss: 0.0062 - val_accuracy: 0.0625
Epoch 4/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0061 - accuracy: 0.1099 - val_loss: 0.0062 - val_accuracy: 0.0748
Epoch 5/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0060 - accuracy: 0.1418 - val_loss: 0.0062 - val_accuracy: 0.0846
Epoch 6/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0059 - accuracy: 0.1786 - val_loss: 0.0062 - val_accuracy: 0.0760
Epoch 7/500
102/102 [==============================] - 1s 9ms/step - loss: 0.0058 - accuracy: 0.2261 - val_loss: 0.0063 - val_accuracy: 0.0821

In [ ]:
print(results)
win = max(results, key=lambda x: results[x][1])
print(f'max on test: {win}: {results[win]}')

{'doubleLSTM': [0.006199141964316368, 0.11274509876966476], 'bidoubleLSTM': [0.0063291145488619804, 0.0], 'hydoublelstm': [0.006293037906289101, 0.10661764442920685], 'lstm': [0.006291882134974003, 0.11642156541347504], 'rnn': [0.006473690737038851, 0.09313725680112839], 'singrnn': [0.006290123797953129, 0.07598039507865906], 'bilstm': [0.00637702364474535, 0.10784313827753067], 'conv': [0.0063360948115587234, 0.09313725680112839], 'clstm': [0.006919065956026316, 0.08088235557079315]}
max on test: hydoublelstm: [0.006293037906289101, 0.10661764442920685]


In [96]:
models = [clstm_model, bi_LSTM_model, lSTM_model,hydoubleLSTM_model,doubleLSTM_model,bidoubleLSTM_model,rnn_model,singrnn_model,conv_model]
losses = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
names = ['clstm', 'bilstm', 'lstm', 'hylstm', 'doubllstm', 'bidoublels', 'rnn', 'singrnn', 'conv']
results = {}
for loss in losses:
    the_loss = loss
    print(loss)
    for model, name in tqdm(zip(models, names)):
        print(name)
        model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])
        model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs, verbose=0)
        results[f'{loss}_{name}']= model.evaluate(X_test, y_test)




mse


0it [00:00, ?it/s]

clstm
13/13 [==============================] - 0s 3ms/step - loss: 0.0066 - accuracy: 0.0748


1it [00:22, 22.39s/it]

bilstm
13/13 [==============================] - 0s 20ms/step - loss: 0.0071 - accuracy: 0.1042


...

7it [06:01, 50.15s/it]

singrnn
13/13 [==============================] - 0s 3ms/step - loss: 0.0877 - accuracy: 0.0515


8it [06:11, 37.25s/it]

conv
13/13 [==============================] - 0s 441us/step - loss: 0.0762 - accuracy: 0.0858


9it [06:12, 41.35s/it]


In [97]:
print(results)
win = max(results, key=lambda x: results[x][1])
print(f'max on test: {win}: {results[win]}')

{'mse_clstm': [0.006649418268352747, 0.07475490123033524], 'mse_bilstm': [0.007117485161870718, 0.1041666641831398], 'mse_lstm': [0.006976249162107706, 0.09803921729326248], 'mse_hylstm': [0.006493318360298872, 0.08088235557079315], 'mse_doubllstm': [0.00662790285423398, 0.08455882221460342], 'mse_bidoublels': [0.0063291145488619804, 0.0], 'mse_rnn': [0.006597792264074087, 0.0674019604921341], 'mse_singrnn': [0.0064606391824781895, 0.053921569138765335], 'mse_conv': [0.006725983694195747, 0.07475490123033524], 'categorical_crossentropy_clstm': [9.013019561767578, 0.02450980432331562], 'categorical_crossentropy_bilstm': [6.715873718261719, 0.022058824077248573], 'categorical_crossentropy_lstm': [7.20968770980835, 0.019607843831181526], 'categorical_crossentropy_hylstm': [8.061545372009277, 0.026960784569382668], 'categorical_crossentropy_doubllstm': [7.012162208557129, 0.014705882407724857], 'categorical_crossentropy_bidoublels': [nan, 0.0], 'categorical_crossentropy_rnn': [7.1701817512

In [98]:
models = [clstm_model, bi_LSTM_model, lSTM_model,hydoubleLSTM_model,doubleLSTM_model,bidoubleLSTM_model,rnn_model,singrnn_model,conv_model]
losses = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
names = ['clstm', 'bilstm', 'lstm', 'hylstm', 'doubllstm', 'bidoublels', 'rnn', 'singrnn', 'conv']
for loss in losses:
    the_loss = loss
    print(loss)
    for model, name in tqdm(zip(models, names)):
        print(name)
        model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])
        model.fit(X_train,y_train, validation_data=(X_val, y_val), epochs=num_epochs, verbose=0)
        results[f'long_{loss}_{name}']= model.evaluate(X_test, y_test)

mse


0it [00:00, ?it/s]

clstm
13/13 [==============================] - 0s 3ms/step - loss: 0.0069 - accuracy: 0.0686


1it [06:51, 411.14s/it]

bilstm
13/13 [==============================] - 0s 19ms/step - loss: 0.0073 - accuracy: 0.0980


...

hylstm
13/13 [==============================] - 0s 22ms/step - loss: 0.0065 - accuracy: 0.0907


4it [2:30:58, 2591.26s/it]

doubllstm


4it [3:01:50, 2727.61s/it]


KeyboardInterrupt: 

In [ ]:
print(results)
win = max(results, key=lambda x: results[x][1])
print(f'max on test: {win}: {results[win]}')